In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import numpy as np
from data_science import CompareData
from data_science.tools import CompressWavelet

In [3]:
from data_science.analysis_methodology import (
    Autoencoder,
    ClusterAnalysisDivergence,
    DeepAnT_CNN,
    HistogramConsistencyTest,
    ProcessingArchitecture,
)

In [6]:
# load data (download https://1drv.ms/u/s!AuAnQXzLmsvjihFVtI28bX0y0OyW?e=dDug9Y)
abs_path = os.path.dirname(os.getcwd()) + "/examples"
data = np.load(abs_path + "/data/data_3.npy")
# get dt
fs = 10000
dt = 1 / fs
#remove time column
data = data[: fs * 6, :4]

In [5]:
histogram = HistogramConsistencyTest(
    nominal_rotation=60, filter_type="frequency", p_value_limit=0.05
)

auto_encoder = Autoencoder(size_sub_pack=512,)

deep_ant = DeepAnT_CNN(epochs=2)

adaptative_cluster = ClusterAnalysisDivergence(
    n_channels=True,
    size_sub_sample=256,
    oversampling=True,
    step_to_overlaps=50,
    discriminators=["rms", "kurtosis", "peak value"],
    nb_cluster=6,
    threshold=None,
    sigma=2.5,
    size_buffer=8,
)

processing_architecture = ProcessingArchitecture(
    [auto_encoder, deep_ant, histogram, adaptative_cluster,]
)

# # start comparison

compare_test = CompareData(data, dt, processing_architecture, slice_size=4000)


Number of samples: 15
0 of 15 samples analyzed...
Iniciando análise da amostra
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, None, 24)          216       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 24)          1176      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 24)          1176      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 24)          1176      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 24)          1176      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 24)          1176      
___________________________________________

In [7]:
import time
# record_array = compare_test.record
record_array = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1]
save_path = abs_path + "/data/wavelet_old"
cw = CompressWavelet(save_path, record_array, save_mode="ALL")

start_time_wvl = time.time()
    
compare_test = CompareData(data, dt, cw, slice_size=fs)
    
spend_time_wvl = time.time() - start_time_wvl


Number of samples: 6
0 of 6 samples analyzed...
sample = 8
sample = 16
sample = 24
sample = 32
sample = 40
sample = 48
6 of 6 samples analyzed!


In [12]:
#Load from file
rec_data = cw.decode_from_file()
rec_data.shape


(60000, 4)

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mse = mean_squared_error(data, rec_data)
mae= mean_absolute_error(data, rec_data)
print("Avaliação do sinal reconstruído:")
print(f"MSE: {mse}, MAE: {mae}")

Avaliação do sinal reconstruído:
MSE: 0.023251261166575957, MAE: 0.07080130221630083
